In [229]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{I} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [230]:
import numpy as np

def householder(vec):
    n = len(vec)
    vec = np.asarray(vec, dtype=float)
    norm = np.linalg.norm(vec)
    u = vec - norm * np.eye(n)[0]

    H = np.eye(n) - 2 * np.outer(u, u) / np.dot(u, u)

    outvec = np.dot(H, vec)

    return  outvec, H

Test your function using tests below:

In [231]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)


In [232]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)


# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

$$
\mathbf{A} = (\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1)^{-1} \mathbf{R} \;,
$$
so 
$$
\mathbf{A} =  \mathbf{Q} \mathbf{R} \;,
$$
with
$$
\mathbf{Q} = (\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1)^{-1} =  \mathbf{H}_1^{-1}  \mathbf{H}_2^{-1}  \cdots \mathbf{H}_{n-1}^{-1} = \mathbf{H}_1^T  \mathbf{H}_2^T  \cdots \mathbf{H}_{n-1}^T 
$$
Since $\mathbf{H}_i$ is ortogonal
$$
\mathbf{H}_i\mathbf{H}_i^T = \mathbf{I}
$$
then
$$
\mathbf{H}^{-1} = \mathbf{H}^T 
$$
but also  $\mathbf{H}_i$ is symetric
$$
\mathbf{H}_i^T = \mathbf{H}_i
$$
so
$$
\mathbf{Q} = \mathbf{H}_1^{-1}  \mathbf{H}_2^{-1}  \cdots \mathbf{H}_{n-1}^{-1} = \mathbf{H}_1^T  \mathbf{H}_2^T  \cdots \mathbf{H}_{n-1}^T =  \mathbf{H}_1 \mathbf{H}_2  \cdots \mathbf{H}_{n-1}
$$


In [71]:
def qr_decomp(a):
  
    m, n = a.shape
    Q = np.eye(m)
    R = a.copy()

    for i in range(min(m, n)):
        x = R[i:, i]
        norm = np.linalg.norm(x)
        if x[0] < 0:
            norm = -norm
        u = x.copy()
        u[0] += norm
        u = u.astype(float)  # Explicitly specify the data type as float
        u /= np.linalg.norm(u)

        H = np.eye(m)
        H[i:, i:] -= 2.0 * np.outer(u, u)
        
        R = np.dot(H, R)
        Q = np.dot(Q, H.T)

    return Q, R

In [72]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [73]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [74]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

`Check if your q and r agree with qq and rr. Explain.`

In [223]:
from scipy.linalg import qr
a = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4], [9,3,1]])
q, r = qr_decomp(a)
qq, rr = qr(a)
print(q)
print(qq)
print(r)
print(rr)
print(a)
print(q@r)
print(qq@rr)

[[-0.22887865 -0.09289517 -0.02407796 -0.42505664 -0.87047812]
 [-0.30721966 -0.57985684  0.34192067  0.64732123 -0.18288687]
 [-0.55299539 -0.4926121  -0.35829868 -0.40048423  0.40344002]
 [-0.49155146  0.48067293 -0.5528284   0.45312486 -0.12802081]
 [-0.55299539  0.42593825  0.66971137 -0.18599011  0.17224129]]
[[-0.24479602 -0.06723049  0.00981821 -0.41631113 -0.87300837]
 [-0.30599503 -0.58266423  0.33762629  0.64971343 -0.17535786]
 [-0.55079106 -0.4964713  -0.36457628 -0.41134781  0.38473703]
 [-0.48959205  0.47923271 -0.55563068  0.4547994  -0.1227505 ]
 [-0.55079106  0.42406923  0.66653641 -0.16884307  0.20979928]]
[[-16.33794163 -10.4562212   -4.40706605]
 [ -0.02554617  -6.53203171   0.77789847]
 [ -0.00662144  -0.          -2.64057622]
 [ -0.11689058   0.          -0.        ]
 [ -0.23938148   0.           0.        ]]
[[-16.34013464 -10.46503005  -4.40632844]
 [  0.          -6.51790964   0.7843557 ]
 [  0.           0.          -2.63989693]
 [  0.           0.           0

Enter your explanation here (10% of the total grade, peer-graded)

**My explanation**

By the way, q and r agree with qq and rr because if the matrices q and qq as well as r and rr are element-wise equal within the specified tolerance, the assert_allclose function will not raise an error, indicating that the matrices agree.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).




Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

$$
\mathbf{R} = \mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} 
$$
and 
$$
\mathbf{H}_i = \mathbf{I} - 2 \mathbf{u}_i \mathbf{u}_i^T
$$
so if
$$
\mathbf{R}_0 =  \mathbf{A}
$$
then 
$$
\mathbf{R}_1 = \mathbf{H}_1 \mathbf{R}_0= \ (\mathbf{I} - 2 \mathbf{u}_1 \mathbf{u}_1^T) \mathbf{R}_0 =  \mathbf{R}_0 -  2 \mathbf{u}_1 ( \mathbf{u}_1^T  \mathbf{R}_0)
$$
and
$$
\mathbf{R}_2 = \mathbf{H}_2 \mathbf{R}_1 = \ (\mathbf{I} - 2 \mathbf{u}_2 \mathbf{u}_2^T) \mathbf{R}_1 =  \mathbf{R}_1 -  2 \mathbf{u}_2 ( \mathbf{u}_2^T  \mathbf{R}_1)
$$
so on until
$$
\mathbf{R} = \mathbf{H}_n \mathbf{R}_{n-1} =  (\mathbf{I} - 2 \mathbf{u}_{n-1} \mathbf{u}_{n-1}^T) \mathbf{R}_{n-1} = \mathbf{R}_{n-1} -  2 \mathbf{u}_{n-1} ( \mathbf{u}_{n-1}^T  \mathbf{R}_{n-1})
$$

In [224]:
def r_decomp(a):
  
    m, n = a.shape
    r = a.astype(float)  # Convert to floating-point type
    vecs = []

    for i in range(n):
        u = r[i:, i].copy()
        u[0] = u[0] + np.sign(u[0]) * np.linalg.norm(u)
        u = u / np.linalg.norm(u)
        r[i:, i:] -= 2.0 * np.outer(u, np.dot(u, r[i:, i:]))

        vecs.append(u)

    return r, vecs

$$
\mathbf{Q} =  \mathbf{H}_1 \mathbf{H}_2  \cdots \mathbf{H}_{n-1}
$$
and
$$
\mathbf{H}_i = \mathbf{I} - 2 \mathbf{u}_i \mathbf{u}_i^T
$$
so 
$$
\mathbf{Q} =  (\mathbf{I} - 2 \mathbf{u}_1 \mathbf{u}_1^T )( \mathbf{I} - 2 \mathbf{u}_2 \mathbf{u}_2^T)  \cdots (\mathbf{I} - 2 \mathbf{u}_{n-1} \mathbf{u}_{n-1}^T)
$$

In [225]:
def q_decomp(vecs):
    m, n = len(vecs[0]), len(vecs)
    Q = np.eye(m)

    for i in range(n-1, -1, -1):
        u = vecs[i]
        outer_product = np.outer(u, np.dot(u.T, Q[i:, :]))
        Q[i:, :] -= 2 * outer_product
        
    return Q, Q.T

In [226]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
r, vecs = r_decomp(a)
q,qt = q_decomp(vecs)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

In [227]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

In [228]:
a = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4], [9,3,1]])
R, vecs = r_decomp(a)
print("R\n ",R)
Q,QT = q_decomp(vecs)
print("Q\n",Q)
print("QT\n",QT)
print("QT@Q\n",QT@Q)
print("a\n",a)
print("Q@R\n",Q@R)

R
  [[-16.34013464 -10.46503005  -4.40632844]
 [  0.          -6.51790964   0.7843557 ]
 [  0.          -0.          -2.63989693]
 [  0.           0.          -0.        ]
 [  0.           0.           0.        ]]
Q
 [[-0.24479602 -0.06723049  0.00981821 -0.41631113 -0.87300837]
 [-0.30599503 -0.58266423  0.33762629  0.64971343 -0.17535786]
 [-0.55079106 -0.4964713  -0.36457628 -0.41134781  0.38473703]
 [-0.48959205  0.47923271 -0.55563068  0.4547994  -0.1227505 ]
 [-0.55079106  0.42406923  0.66653641 -0.16884307  0.20979928]]
QT
 [[-0.24479602 -0.30599503 -0.55079106 -0.48959205 -0.55079106]
 [-0.06723049 -0.58266423 -0.4964713   0.47923271  0.42406923]
 [ 0.00981821  0.33762629 -0.36457628 -0.55563068  0.66653641]
 [-0.41631113  0.64971343 -0.41134781  0.4547994  -0.16884307]
 [-0.87300837 -0.17535786  0.38473703 -0.1227505   0.20979928]]
QT@Q
 [[ 1. -0. -0.  0.  0.]
 [-0.  1.  0.  0.  0.]
 [-0.  0.  1.  0. -0.]
 [ 0.  0.  0.  1. -0.]
 [ 0.  0. -0. -0.  1.]]
a
 [[4 3 1]
 [5 7 0]
 [9